# Chapter 9 - A Simplistic Introduction to Algorithmic Complexity

Wriring efficient program is not easy. The most straightforward solution is often not the most efficient. Computationally efficient algorithms often employ subtle tricks that can make them difficult to understand.Consequently, programmers often increase the conceptual complexity of a program in an effort to reduce its computational complexity. To do this in a sensible way, we need to understand how to go about estimating the computational complexity of a program .

## 9.1 Thinking About Computational Complexity

How should one go about answering the question "How long will the following function take to run ?"

In [ ]:
def f(i):
    """Assumes that i>=0 is an integer."""
    ans = 1
    while i>=1:
        ans *= i
        i -= 1
    return ans

In [ ]:
f(5)

If we run the program on some particular input and time it, then it would not be informative because the result would depend upon:

* the speed of the computer on which it is run.
* the efficiency of the Python implementation on that machine.
* the value of the input.

We can get around the first two issues by using a more abstract measure of time. We measure time in terms of the number of basic steps executed by the program. 

For simplicity, we will use a random access machine as our model of computation. In a random acccess machine, steps are executed sequentially, one at a time. A step is an operation that takes a fixed amount of time, such as binding a variable to an object, making a comparison, executing an arithmetic operation, or accessing an object in memory.

Now we turn to the question of dependence on the value of the input. We deal with that by moving away from expressing time complexity as a single number and instead relating it to the sizes of the inputs. This allow us to compare the efficiency of two algorithms by talking about how the running time of each grows with respect to the sizes of the inputs.

Of course, the actual running time of an algorithm depends not only upon the sizes of the inputs but also upon their values. Consider, for example, the linear search algorithm implemented by : 

In [ ]:
def linearSearch(L,x):
    for e in L:
        if e == x:
            return True
    return False

In [ ]:
L = [1,2,3,4,5]
x = 9

linearSearch(L,x)

Suppose that L is a list containing a million elements, and consider the call
linearSearch(L, 3). If the first element in L is 3, linearSearch will return True almost immediately. On the other hand, if 3 is not in L, linearSearch will have to
examine all one million elements before returning False.

In general, there are three broad cases to think about:

* The best-case running time is the running time of the algorithm when the inputs are as favorable as possible. I.e. the best case running-time is the minimum running time over all the possible inputs of a given size. For linearSearch, the best-case running time is independent of the size of L.
* Similarly, the worst-case running time is the maximum running time over all the possible inputs of a given size. For linearSearch, the worst-case running time is linear in the size of L.
* The average-case (expected-case) running time is the average running-time over all possible inputs of a given size. Alternatively, if one has some a priori information about the distribution of input values (e.g. tha 90% of the time x is in L), one can take that into account.

People usually focus on the worst-case. The worst-case provide an upper bound on the running time. This is critical in situations where there is a time constraints on how long a computation can take.

Let's look at the worst-case running time of an iterative implementation of the factorial function:

In [ ]:
def fact(n):
    """Assumes n>0 is natural number. Return n! ."""
    ans = 1      #1 step
    while n>1:   #n-times iteration
        ans *= n #2 steps
        n -= 1   #2 steps 
    return ans   #1 step

In [ ]:
fact(5)

The number of steps required to run fact(n) is something like 2+5n. So, for example, if n=1000, the function will execute roughly  5002 steps. 

It shoud be obvious that as n gets large, worrying about the difference between 5n and 5n+2 is a kind of silly. For this reason, we typically ignore additive constants when reasoning about running time. Multiplicative factor, however, can be important.

On the other hand, when one is comparing two different algorithm, it is often the case that even multiplicative constants are irrelevant. Recall two algorithms, exhaustive enumeration and bisection search, for finding an approximation to the square root of a floating point number. 

In [1]:
#Using exhaustive enumeration to approximate square root.
def squareRootExhaustive(x,epsilon):
    """Assumes x>0 and 0<epsilon<1 are floats. 
       Return a y s.t. y*y is within epsilon of x."""
    step = epsilon**2
    ans = 0.0
    numIter = 0
    
    while abs(ans**2-x) >= epsilon and ans**2 <= x :
        ans += step
        numIter += 1
    if ans**2 > x:
        raise ValueError
    return numIter, ans

In [2]:
#Using bisection search to approximate square root.
def squareRootBi(x,epsilon):
    """Assumes x>0 and 0<epsilon<1 are floats. 
       Return a y s.t. y*y is within epsilon of x."""
    low = 0.0
    high = max(1.0,x)
    ans = (low + high)/2.0
    numIter = 0
    
    while abs(ans**2-x) >= epsilon :
        if ans**2 < x:
            low = ans
        else:
            high = ans
        ans = (low + high)/2.0
        numIter += 1
    return numIter, ans

In [5]:
squareRootExhaustive(100, 0.01)

(99995, 9.999499999990034)

In [4]:
squareRootBi(100, 0.01)

(16, 9.999847412109375)

We see that exhaustive enumeration was so slow as to be impractical for many combinations of x and epsilon. When the difference in the number of iteration is this large, it doesn't really matter how many instructions are in the loop, i.e. the multiplicative constants are irrelevant.

## 9.2 Asymptotic Notation

The asymptotic notation is used to provide a formal way to talk about the relationship between the running time of an algorithm and the size of its inputs. The underlying motivation is that almost any algorithm is sufficiently efficient when run on small inputs. What we typically need to worry about is the efficiency of the algorithm when run on very large inputs. As a proxy for "very large", asymptotic notation describes the complexity of an algorithm as the size of its inputs approaches infinity.

In [9]:
def f(x):
    """Assumes that x>0 is an integer."""
    ans = 0
    
    #Loop that takes constant time.
    for i in range(1000):
        ans += 1
    print('Number of additions so far', ans)
    
    #Loop that takes time x.
    for i in range(x):
        ans += 1
    print('Number of additions so far', ans)
    
    #Nested loop take time x**2
    for i in range(x):
        for j in range(x):
            ans += 1
            ans += 1
    print('Number of additions so far', ans)

If one assumes that each line of code takes one unit of time to execute, the running time of this function can be described as 1000+x+2x^2. 

In [10]:
f(10)

Number of additions so far 1000
Number of additions so far 1010
Number of additions so far 1210


In [12]:
f(1000)

Number of additions so far 1000
Number of additions so far 2000
Number of additions so far 2002000


For small values of x the constant term dominates. However, for large x the square term dominates. This kind of analysis lead us to use the following rules of thumb in describing the asymptotic complexity of an algorithm:

* If the running time is the sum of multiple terms, keep the one with the largest growth rate and drop the others.
* If the remaining term is a product, drop any constants.

The most commonly used asymptotic notation is called "Big O" notation. Big O notation is used to give an upper bound on the asymptotic growth of a function. For example, the formula f(x) is an element of O(x^2) means that the function f grows no faster than the quadratic polynomial x^2, in an asymptotic sense.

We will often abuse Big O notation by making statements like "the complexity of f(x) is O(x^2)". By this we mean that in the worst case f will take O(x^2) steps to run. The difference between a function being "in O(x^2)" and "being O(x^2)" is subtle but important. Saying that f(x) in O(x^2) does not preclude the worst-case running time of f from being considerably less than O(x^2).

When we say that f(x) is O(x^2), we are implying that x^2 is both an upper and a lower bound on the asymptotic worst-case running time. This is called tight bound.

## 9.3 Some Important Complexity Classes

Some of the most common instances of Big O are listed below. In each case, n is a measure of the size of the inputs to the function.

*  O(1) denotes constant running time.
* O(log	n) denotes logarithmic running time.
* O(n) denotes linear running time.
* O(n	log	n) denotes log-linear running time.
* O(n^k) denotes polynomial running time. Notice that k is a constant.
* O(c^n) denotes exponential running time. Here a constant is being raised to a
power based on the size of the input.

### 9.3.1 Constant Complexity

This indicates that the asymptotic complexity is independent of the size of the
inputs. There are very few interesting programs in this class, but all programs
have pieces (for example finding out the length of a Python list or multiplying
two floating point numbers) that fit into this class. Constant running time does
not imply that there are no loops or recursive calls in the code, but it does imply
that the number of iterations or recursive calls is independent of the size of the
inputs.

### 9.3.2 Logarithmic Complexity

Such function have a complexity that grows as the log of at least one of the inputs. We don't care about the base of the log, since the difference between one base and another is merely a constant multiplicative factor. Consider:

In [37]:
def intToStr(i):
    """Assumes i>=0 is an integer. 
    Returns a decimal string representation of i."""
    digits = '0123456789'
    if i == 0:
        return '0'
    result = ''
    while i > 0:
        result = digits[i%10] + result
        i = i//10
    return result

In [38]:
intToStr(68)

'68'

Since there are no function or method calls in this code, we know that we only have to look at the loops to determine the complexity class. There is only one loop, so the only thing we need to do is characterize the number of iterations. That boils down to the number of times we can use integer division to divide i by 10 before getting result of 0. So, the complexity of intToStr is O(log(i)). What about the complexity of :

In [40]:
def addDigits(n):
    """Assumes n>=0 is an integer.
       Returns the sum of the digits in n."""
    stringRep = intToStr(n) #string rep. of a given integer with complexity O(log(n))
    val = 0
    for c in stringRep:  
        val += int(c)    
    return val

In [43]:
val = '1abc5'

for i in val:
    print(i)

1
a
b
c
5


The complexity of converting n to a string using intToStr is O(log(n)) and intToStr returns a string of length O(log(n)). The for loop will be executed O(len(stringRep)) times, i.e. O(log(n)) times. Assume that a character can be converted to a digitin constant time, the program will run in time proportional to O(log(n)) + O(log(n)), which makes it O(log(n)).

### 9.3.3 Linear Complexity

Many algorithms that deal with list or other kinds of sequences are linear because they touch each element of the sequence a constant (greater than 0) number of times. Consider, for example:

In [44]:
def addDigits(s):
    """Assumes s is a str, each character of which is a decimal digit. 
       Return an integer that is the sum of the digits in s."""
    val = 0
    for c in s:
        val += int(c)
    return val

In [45]:
addDigits('12345')

15

Thus function is linear in the length of s, i.e. O(len(s)), again assuming that a character representing a digit can be converted to an integer in constant time. Of course, a program does not need to have a loop to have linear complexity. Consider:

In [1]:
def factorial(x):
    """Assumes that x>0 is an integer. Returns x! ."""
    if x == 1:
        return 1
    else:
        return x*factorial(x-1)

There are no loops in this code, so in order to analyze the complexity we need to figure out how many recursive calls get made. The series of calls is simply:

    factorial(x), factorial(x-1), ..., factorial(2), factorial(1)
    
The length of this series, and thus the complexity of the function, is O(x).

### 9.3.4 Log-Linear Complexity

This is slightly more complicated than the complexity classes we have looked at
thus far. It involves the product of two terms, each of which depends upon the
size of the inputs. It is an important class, because many practical algorithms are
log-linear. The most commonly used log-linear algorithm is probably merge sort,
which is O(n	log(n)), where n is the length of the list being sorted. We will look at
that algorithm and analyze its complexity in Chapter 10.

### 9.3.5 Polynomial Complexity

The most commonly used polynomial algorithms are quadratic, i.e. their complexity grows as the square of the size of their input. Consider, for example the following function which implement a subset test.

In [5]:
def isSubset(L1,L2):
    """Assumes L1 and L2 are lists. Returns True if each 
       element in L1 is also in L2, and False otherwise."""
    for e1 in L1:
        matched = False
        for e2 in L2:
            if e1 == e2:
                matched = True
                break
        if not matched:
            return False
    return True

In [6]:
isSubset([1,2,3],[4,5,6,1,2,3,7,8,9])

True

Each time the inner loop is reached it is executed O(len(L2) times. The function isSubset will execute the outer loop O(len(L1)) times, so the inner loop will
be reached O(len(L1)) times. Therefore, the complexity of the function isSubset
is O(len(L1)* len(L2)). Now consider the function intersect:

In [9]:
def intersect(L1,L2):
    """Assumes both L1 and L2 are lists. Return a list without duplicates that is the intersection of L1 and L2."""
    
    #Build a list containing common elements.
    tmp = []
    for e1 in L1:
        for e2 in L2:
            if e1 == e2:
                tmp.append(e1)
                break
    
    #Build a list without duplicates
    result  = []
    for e in tmp:
        if e not in result:
            result.append(e)
    return result

The running time for the part building the list that might contain duplicates
is clearly O(len(L1)* len(L2)). At first glance, it appears that the part of the code
that builds the duplicate-free list is linear in the length of tmp, but it is not. The
test e not in result potentially involves looking at each element in result, and is
therefore O(len(result)); consequently the second part of the implementation is
O(len(tmp)* len(result)). However, since the lengths of result and tmp are bounded by the length of the smaller of L1 and L2, and since we ignore additive terms,
the complexity of intersect is O(len(L1)*len(L2)).

### 9.3.6 Exponential Complexity

Many importants problems are inherently exponential, i.e. solving them completely can require time that is exponential inthe size of the input. This is unfortunate, since it rarely pays to write a program that has a reasonably high probability of taking exponential time to run. Consider, for example:

In [13]:
def getBinaryRep(n, numDigits):
    """Assumes n and numDigits are non-negative integers. Returns
       a str of length numDigits that is a binary rep. of n."""
    
    result = ''
    while n>0: #O(n)
        result = str(n%2) + result #either 0 (n is even) or 1 (n is odd).
        n = n//2 #stair case division
    if len(result) > numDigits:
        raise ValueError('not enough digits.')
    for i in range(numDigits-len(result)): #O(numDigits)
        result = '0' + result
    return result

def genPowerset(L):
    """Assumes L is a list. Returns a list of lists that contains all possible 
       combinations of the elements of L.  E.g., if L is [1, 2] it will return 
       a list with elements [], [1], [2], and [1,2]"""
    powerset = []
    for i in range(0,2**len(L)): #O(2**len(L))
        binStr = getBinaryRep(i, len(L))
        subset = []
        for j in range(len(L)): #O(len(L))
            if binStr[j] == '1':
                subset.append(L[j])
        powerset.append(subset)
    return powerset

In [16]:
L = ['a','b','c','d','e','f','g','h','i','j']
genPowerset([L])

[[], [['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']]]

The function genPowerset(L) returns a list of lists that contains all possible
combinations of the elements of L. For example, if L is ['x', 'y'], the powerset of
L will be a list containing the lists [], ['x'], ['y'], and ['x', 'y'].
The algorithm is a bit subtle. Consider a list of n elements. We can represent
any combination of elements by a string of n 0’s and 1’s, where a 1 represents the
presence of an element and a 0 its absence. The combination containing no items
is represented by a string of all 0’s, the combination containing all of the items is
represented by a string of all 1’s, the combination containing only the first and
last elements is represented by 100…001, etc

Generating all sublists of a list L of length n can be done as follows:

* Generate all n-bit binary numbers. These are the numbers from 0 to 2n.
* For each of these 2n +1 binary numbers, b, generate a list by selecting those
elements of L that have an index corresponding to a 1 in b. For example, if L is
['x', 'y'] and b is 01, generate the list ['y'].

Try running genPowerset on a list containing the first ten letters of the alphabet. It will finish quite quickly and produce a list with 1024 elements. Next, try
running genPowerset on the first twenty letters of the alphabet. It will take more
than a bit of time to run, and return a list with about a million elements. If you
try running genPowerset on all twenty-six letters, you will probably get tired of
waiting for it to complete, unless your computer runs out of memory trying to
build a list with tens of millions of elements. Don’t even think about trying to run
genPowerset on a list containing all uppercase and lowercase letters. Step 1 of the
algorithm generates O(2len(L)) binary numbers, so the algorithm is exponential in
len(L).

Does this mean that we cannot use computation to tackle exponentially hard
problems? Absolutely not. It means that we have to find algorithms that provide
approximate solutions to these problems or that find perfect solutions on some
instances of the problem. But that is a subject for later chapters.

### 9.3.7 Comparisons of Complexity Classes

The plots in this section are intended to convey an impression of the implications
of an algorithm being in one or another of these complexity classes.
The plot on the left in Figure 9.7 compares the growth of a constant-time algorithm to that of a logarithmic algorithm. Note that the size of the input has to
reach about a million for the two of them to cross, even for the very small constant of twenty. When the size of the input is five million, the time required by a
logarithmic algorithm is still quite small. The moral is that logarithmic algorithms are almost as good as constant-time ones.
The plot on the right of Figure 9.7 illustrates the dramatic difference between logarithmic algorithms and linear algorithms. Notice that the x-axis only
goes as high as 1000. While we needed to look at large inputs to appreciate the
difference between constant-time and logarithmic-time algorithms, the difference between logarithmic-time and linear-time algorithms is apparent even on
small inputs. The dramatic difference in the relative performance of logarithmic
and linear algorithms does not mean that linear algorithms are bad. In fact, most
of the time a linear algorithm is acceptably efficient.

![](97growth.jpg)

The plot on the left in Figure 9.8 shows that there is a significant difference
between O(n) and O(n	log(n)). Given how slowly log(n) grows, this may seem a
bit surprising, but keep in mind that it is a multiplicative factor. Also keep in
mind that in many practical situations, O(n	log(n)) is fast enough to be useful.
On the other hand, as the plot on the right in Figure 9.8 suggests, there are many
situations in which a quadratic rate of growth is prohibitive.

![](98growth.jpg)

The plots in Figure 9.9 are about exponential complexity. In the plot on the
left of Figure 9.9, the numbers to the left of the y-axis run from 0.0 to 1.2. However, the notation x1e301 on the top left means that each tick on the y-axis should
be multiplied by 10301. So, the plotted y-values range from 0 to roughly 1.1* 10301.But it looks almost as if there are no curves in the plot on the left in Figure 9.9.
That’s because an exponential function grows so quickly that relative to the y
value of the highest point (which determines the scale of the y-axis), the y values
of earlier points on the exponential curve (and all points on the quadratic curve)
are almost indistinguishable from 0.
The plot on the right in Figure 9.9 addresses this issue by using a logarithmic
scale on the y-axis. One can readily see that exponential algorithms are impractical for all but the smallest of inputs.
Notice that when plotted on a logarithmic scale, an exponential curve appears as a straight line. We will have more to say about this in later chapters.

![](99growth.jpg)